### Problem 1: Sum of a Range of Integers

#### Solution 1: Real Python

In [1]:
""" Sum of Integers Up To n
    Write a function, add_it_up(), that takes a single integer as input
    and returns the sum of the integers from zero to the input parameter.

    The function should return 0 if a non-integer is passed in.
"""
def add_it_up(n):
    try:
        acc = sum(range(n + 1))
    except TypeError:
        acc = 0
    return acc

add_it_up(10)

55

#### Solution 2: Tail Recursive

In [2]:
def add_it_up_recursive(n, acc = 0):
    if n <= 0:
        return acc
    return add_it_up_recursive(n - 1, acc + n)

add_it_up_recursive(10)

55

### Problem 2: Caesar Cipher

#### Solution 1: Real Python

In [3]:
""" A caesar cipher is a simple substitution cipher where each letter of the
    plain text is substituted with a letter found by moving 'n' places down the
    alphabet. For an example, if the input plain text is:

        abcd xyz

    and the shift value, n, is 4. The encrypted text would be:

        efgh bcd

    You are to write a function which accepts two arguments, a plain-text
    message and a number of letters to shift in the cipher. The function will
    return an encrypted string with all letters being transformed while all
    punctuation and whitespace remains unchanged.

    Note: You can assume the plain text is all lowercase ascii, except for
    whitespace and punctuation.
    
    For this problem, you’re free to use any part of the Python standard library to do the transform.
"""

import string

def caesar(plain_text, shift_num=1):
    letters = string.ascii_lowercase
    mask = letters[shift_num:] + letters[:shift_num]
    trantab = str.maketrans(letters, mask)
    return plain_text.translate(trantab)

caesar('abcd xyz', 4)

'efgh bcd'

#### Solution 2: Mine - will work even if shift_num > 26

In [4]:
def caesar_mine(plain_text, shift_num=1):
    translation_dict = {ord('a') + i : ord('a') + (i + shift_num) % 26 for i in range(26)}
    return plain_text.translate(translation_dict)

caesar_mine('abcd xyz', 4)

'efgh bcd'

### Problem 3: Caesar Cipher Redux

#### Solution 1: Real Python

In [5]:
""" solve the Caesar cipher again, but this time we'll do it without using .translate().
"""

import string

def shift_n(letter, table):
    try:
        index = string.ascii_lowercase.index(letter)
        return table[index]
    except ValueError:
        return letter

def caesar(plain_text, shift_num=1):
    shift_num = shift_num % 26
    table = string.ascii_lowercase[shift_num:] + string.ascii_lowercase[:shift_num]
    enc_list = [shift_n(letter, table) for letter in plain_text]
    return "".join(enc_list)

caesar('abcd xyz', 4)

'efgh bcd'

#### Solution 2: Mine

In [6]:
import string

def caesar_mine(plain_text, shift_num=1):
    letters = string.ascii_lowercase
    translation_dict = {letters[i] : chr(ord('a') + (i + shift_num) % 26) for i in range(26)}
    return ''.join([translation_dict.get(character, character) for character in plain_text])

caesar_mine('abcd xyz', 4)

'efgh bcd'

### Problem 4: Log Parser

#### Solution 1: Real Python

In [7]:
""" log parser
    Accepts a filename on the command line. The file is a Linux-like log file
    from a system you are debugging. Mixed in among the various statements are
    messages indicating the state of the device. They look like this:
        Jul 11 16:11:51:490 [139681125603136] dut: Device State: ON
    The device state message has many possible values, but this program cares
    about only three: ON, OFF, and ERR.

    Your program will parse the given log file and print out a report giving
    how long the device was ON and the timestamp of any ERR conditions.
"""

# Tip: log files can be huge, so don't read the entire file, parse it line by line

import datetime
import sys

def get_next_event(filename):
    with open(filename, "r") as datafile:
        for line in datafile:
            if "dut: Device State: " in line:
                line = line.strip()
                # Parse out the action and timestamp
                action = line.split()[-1]
                timestamp = line[:19]
                yield (action, timestamp)

def compute_time_diff_seconds(start, end):
    format = "%b %d %H:%M:%S:%f"
    start_time = datetime.datetime.strptime(start, format)
    end_time = datetime.datetime.strptime(end, format)
    return (end_time - start_time).total_seconds()

def extract_data(filename):
    time_on_started = None
    errs = []
    total_time_on = 0

    for action, timestamp in get_next_event(filename):
        # First test for errs
        if "ERR" == action:
            errs.append(timestamp)
        elif ("ON" == action) and (not time_on_started):
            time_on_started = timestamp
        elif ("OFF" == action) and time_on_started:
            time_on = compute_time_diff_seconds(time_on_started, timestamp)
            total_time_on += time_on
            time_on_started = None
    return total_time_on, errs

if __name__ == "__main__":
    total_time_on, errs = extract_data('./data/test.log')
    print(f"Device was on for {total_time_on} seconds")
    if errs:
        print("Timestamps of error events:")
        for err in errs:
            print(f"\t{err}")
    else:
        print("No error events found.")

Device was on for 6.305999999999999 seconds
Timestamps of error events:
	Jul 11 16:11:54:661
	Jul 11 16:11:56:067


### Problem 5: Sudoku Solver

#### Solution 1: Real Python

In [8]:
""" Sudoku Solver
    Note: A description of the sudoku puzzle can be found at:

        https://en.wikipedia.org/wiki/Sudoku

    Given a string in SDM format, described below, write a program to find and
    return the solution for the sudoku puzzle in the string. The solution should
    be returned in the same SDM format as the input.

    Some puzzles will not be solvable. In that case, return the string
    "Unsolvable".

    The general SDM format is described here:

        http://www.sudocue.net/fileformats.php

    For our purposes, each SDM string will be a sequence of 81 digits, one for
    each position on the sudoku puzzle. Known numbers will be given, and unknown
    positions will have a zero value.

    For example, assume you're given this string of digits (split into two lines
    for readability):

        0040060790000006020560923000780610305090004
             06020540890007410920105000000840600100

    The string represents this starting sudoku puzzle:

             0 0 4   0 0 6   0 7 9
             0 0 0   0 0 0   6 0 2
             0 5 6   0 9 2   3 0 0

             0 7 8   0 6 1   0 3 0
             5 0 9   0 0 0   4 0 6
             0 2 0   5 4 0   8 9 0

             0 0 7   4 1 0   9 2 0
             1 0 5   0 0 0   0 0 0
             8 4 0   6 0 0   1 0 0

    The provided unit tests may take a while to run, so be patient.
"""

import copy

def line_to_grid(values):
    grid = []
    line = []
    for index, char in enumerate(values):
        if index and index % 9 == 0:
            grid.append(line)
            line = []
        line.append(int(char))
    # Add the final line
    grid.append(line)
    return grid

def grid_to_line(grid):
    line = ""
    for row in grid:
        r = "".join(str(x) for x in row)
        line += r
    return line

def small_square(x, y):
    upperX = ((x + 3) // 3) * 3
    upperY = ((y + 3) // 3) * 3
    lowerX = upperX - 3
    lowerY = upperY - 3
    for subX in range(lowerX, upperX):
        for subY in range(lowerY, upperY):
            # If subX != x or subY != y:
            if not (subX == x and subY == y):
                yield subX, subY

def compute_next_position(x, y):
    nextY = y
    nextX = (x + 1) % 9
    if nextX < x:
        nextY = (y + 1) % 9
        if nextY < y:
            return (True, 0, 0)
    return (False, nextX, nextY)

def test_and_remove(value, possible):
    if value != 0 and value in possible:
        possible.remove(value)


def detect_possible(grid, x, y):
    if grid[x][y]:
        return [grid[x][y]]

    possible = set(range(1, 10))
    # Test horizontal and vertical
    for index in range(9):
        if index != y:
            test_and_remove(grid[x][index], possible)
        if index != x:
            test_and_remove(grid[index][y], possible)

    # Test in small square
    for subX, subY in small_square(x, y):
        test_and_remove(grid[subX][subY], possible)

    return list(possible)

def solve(start, x, y):
    temp = copy.deepcopy(start)
    while True:
        possible = detect_possible(temp, x, y)
        if not possible:
            return False

        finished, nextX, nextY = compute_next_position(x, y)
        if finished:
            temp[x][y] = possible[0]
            return temp

        if len(possible) > 1:
            break
        temp[x][y] = possible[0]
        x = nextX
        y = nextY

    for guess in possible:
        temp[x][y] = guess
        result = solve(temp, nextX, nextY)
        if result:
            return result
    return False

def sudoku_solve(input_string):
    grid = line_to_grid(input_string)
    answer = solve(grid, 0, 0)
    if answer:
        return grid_to_line(answer)
    else:
        return "Unsolvable"

sdm = '016400000200009000400000062070230100100000003003087040960000005000800007000006820'
sudoku_solve(sdm)

'316452978285679314497318562879234156142965783653187249968721435521843697734596821'

#### Solution 2: Mine

In [9]:
import numpy as np
import pandas as pd

def df_to_sdm(df):
    return ''.join([ele for row in df.values.tolist() for ele in row])

def is_feasible_or_solution(df):
    if df.isin(['']).sum().sum() != 0:
        return False, False
    elif df.applymap(lambda ele: len(ele)).sum().sum() == 81:
        return True, True
    return True, False

def suggest_digits(row, column, box, full_set):
    row_set = set(row)
    column_set = set(column)
    box_set = {col_ele for row_ele in box.values.tolist() for col_ele in row_ele}
    return ''.join(list(full_set - row_set.union(column_set).union(box_set)))

def fill_blank_cells_with_possible_values(original_sudoku, full_set):
    run = True
    while run:
        sudoku = original_sudoku.copy()
        run = False
        for i in range(9):
            for j in range(9):
                box_coor_top_left = 3 * (i // 3), 3 * (j // 3)
                if original_sudoku.iloc[i, j] == '0':
                    possible_values = suggest_digits(
                        original_sudoku.iloc[i, :],
                        original_sudoku.iloc[:, j],
                        original_sudoku.iloc[box_coor_top_left[0]:box_coor_top_left[0]+3, box_coor_top_left[1]:box_coor_top_left[1]+3],
                        full_set
                    )
                    if len(possible_values) == 1:
                        run = True
                        original_sudoku.iloc[i, j] = possible_values[0]
                    else:
                        sudoku.iloc[i, j] = possible_values
    return sudoku

def suggest_for_exploration(df):
    mask = df.applymap(lambda ele: len(ele)) > 1
    for i in range(9):
        for j in range(9):
            if mask.iloc[i, j]:
                mask.iloc[i, j] = False
                df[mask] = '0'
                sudoku = df.copy()
                sudoku.iloc[i, j] = df.iloc[i, j][0]
                sudoku[mask] = '0'
                sudoku_explore_later = []
                for digit in df.iloc[i, j][1:]:
                    temp = df.copy()
                    temp.iloc[i, j] = digit
                    sudoku_explore_later.append(temp)
                return(sudoku, sudoku_explore_later)

def sudoku_solve(input_string):
    stack = []
    original_sudoku = pd.DataFrame(np.array(list(input_string)).reshape(9,9))
    full_set = set(map(str, range(1,10)))
    sudoku = original_sudoku.copy()
    while True:
        sudoku = fill_blank_cells_with_possible_values(sudoku, full_set)
        is_feasible, is_solution = is_feasible_or_solution(sudoku)
        if is_solution:
            return df_to_sdm(sudoku)
        elif is_feasible:
            sudoku, sudoku_explore_later = suggest_for_exploration(sudoku)
            stack.extend(sudoku_explore_later)
        elif len(stack) >= 1:
            sudoku = stack.pop()
        else:
            return 'Unsolvable'

sdm = '016400000200009000400000062070230100100000003003087040960000005000800007000006820'
sudoku_solve(sdm)

'316452978285679314497318562879234156142965783653187249968721435521843697734596821'